In [1]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from skimage.transform import resize
from skimage.io import imread

import os
import matplotlib.pyplot as plt

import pickle

In [2]:
datadir = 'Images/'
categories = ['ChickenHead', 'ElephantHead']

flat_data_arr = []
target_arr = []

for i in categories:
    print ('loading... category: ', i)
    path = os.path.join (datadir, i) 
    for img in os.listdir (path):
        img_array = imread (os.path.join (path, img))
        img_resized = resize (img_array, (150, 150, 3))
        flat_data_arr.append (img_resized.flatten())
        target_arr.append (categories.index(i))
    print ("loaded category ", i, "successfully")

loading... category:  ChickenHead
loaded category  ChickenHead successfully
loading... category:  ElephantHead
loaded category  ElephantHead successfully


In [3]:
flat_data = np.array (flat_data_arr)
target = np.array (target_arr)

df = pd.DataFrame (flat_data)
df['Target'] = target

df.head()

,0,1,2,3,4,5,6,7,8,9,...,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target
0,0.258824,0.258824,0.219608,0.231373,0.231373,0.192157,0.168627,0.168627,0.129412,0.180392,...,0.321569,0.321569,0.290196,0.301961,0.301961,0.270588,0.290196,0.290196,0.258824,0
1,0.121569,0.227451,0.105882,0.129412,0.227451,0.113725,0.176471,0.254902,0.149020,0.219608,...,0.784314,0.670588,0.513725,0.772549,0.658824,0.501961,0.768627,0.654902,0.498039,0
2,0.415686,0.698039,0.258824,0.384314,0.666667,0.235294,0.349020,0.619608,0.203922,0.321569,...,0.423529,0.615686,0.266667,0.435294,0.627451,0.270588,0.380392,0.572549,0.215686,0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,0.784314,0.729412,0.623529,0.776471,0.721569,0.615686,0.737255,0.682353,0.576471,0.690196,...,0.717647,0.643137,0.517647,0.725490,0.650980,0.525490,0.721569,0.647059,0.521569,0


In [4]:
X = df.values[:, :-1]
Y = df.values[:, -1]

xtrain, xtest, ytrain, ytest = train_test_split (X, Y, test_size=0.2, random_state=47)

## SVM

In [15]:
param_grid = {'C': [10, 100], 'gamma': [0.0001], 'kernel': ['rbf']}
svc = SVC()

model = GridSearchCV (svc, param_grid)
model.fit (xtrain, ytrain)


GridSearchCV(estimator=SVC(),
             param_grid={'C': [10, 100], 'gamma': [0.0001], 'kernel': ['rbf']})

In [16]:
print (model.best_params_)
ypred_train = model.predict (xtrain)
ypred_test = model.predict (xtest)

print ('Train Accuracy: ', accuracy_score (ypred_train, ytrain))
print ('Test Accuracy: ', accuracy_score (ypred_test, ytest))

{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
Train Accuracy:  1.0
Test Accuracy:  0.9


In [10]:
pickle.dump (model, open('supportVectorMachine.pkl', 'wb'))

In [17]:
model = pickle.load (open('supportVectorMachine.pkl', 'rb'))

In [19]:
filename = 'testImages/chicken1.png'

img_array = imread (filename)
img_resized = resize (img_array, (150, 150, 3))
test_data = np.array (img_resized.flatten())
test_data = test_data.reshape(1, -1)
print (test_data.shape)

(1, 67500)


In [20]:
p = model.predict (test_data)
if p==0: print('chickenHead')
elif p==1: print('elephantHead')

chickenHead


## Random Forest

In [57]:
param_grid = {'n_estimators': [100, 200, 300]}
rf = RandomForestClassifier()

model = GridSearchCV (rf, param_grid)
model.fit (xtrain, ytrain)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [100, 200, 300]})

In [59]:
print (model.best_params_)
ypred_train = model.predict (xtrain)
ypred_test = model.predict (xtest)

print ('Train Accuracy: ', accuracy_score (ypred_train, ytrain))
print ('Test Accuracy: ', accuracy_score (ypred_test, ytest))

{'n_estimators': 200}
Train Accuracy:  1.0
Test Accuracy:  0.9


In [60]:
pickle.dump (model, open('randomForestModel.pkl', 'wb'))

In [35]:
model = pickle.load (open('randomForestModel.pkl', 'rb'))

In [61]:
filename = 'testImages/elephent2.jpg'

img_array = imread (filename)
img_resized = resize (img_array, (150, 150, 3))
test_data = np.array (img_resized.flatten())
test_data = test_data.reshape(1, -1)
print (test_data.shape)

(1, 67500)


In [62]:
p = model.predict (test_data)
if p==0: print('chickenHead')
elif p==1: print('elephantHead')

elephantHead


## Adaboost

In [100]:
ad = AdaBoostClassifier(n_estimators=100)
ad.fit (xtrain, ytrain)

AdaBoostClassifier(n_estimators=100)

In [101]:
ypred_train = ad.predict (xtrain)
ypred_test = ad.predict (xtest)

print ('Train Accuracy: ', accuracy_score (ypred_train, ytrain))
print ('Test Accuracy: ', accuracy_score (ypred_test, ytest))

Train Accuracy:  1.0
Test Accuracy:  0.925


In [102]:
pickle.dump (ad, open('AdaBoostModel.pkl', 'wb'))

In [103]:
ad = pickle.load (open('AdaBoostModel.pkl', 'rb'))

In [104]:
filename = 'testImages/chicken2.jpg'

img_array = imread (filename)
img_resized = resize (img_array, (150, 150, 3))
test_data = np.array (img_resized.flatten())
test_data = test_data.reshape(1, -1)
print (test_data.shape)

(1, 67500)


In [105]:
p = ad.predict (test_data)
if p==0: print('chickenHead')
elif p==1: print('elephantHead')

chickenHead


## KNN

In [5]:
k_range = range(1, 12)
param_grid = {'n_neighbors': k_range}
knn = KNeighborsClassifier()

model = GridSearchCV (knn, param_grid)
model.fit(xtrain, ytrain)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 12)})

In [6]:
print (model.best_params_)
ypred_train = model.predict (xtrain)
ypred_test = model.predict (xtest)

print ('Train Accuracy: ', accuracy_score (ypred_train, ytrain))
print ('Test Accuracy: ', accuracy_score (ypred_test, ytest))

{'n_neighbors': 4}
Train Accuracy:  0.9625
Test Accuracy:  0.875


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

In [7]:
pickle.dump (model, open('KNN_Model.pkl', 'wb'))

In [92]:
model = pickle.load (open('KNN_Model.pkl', 'rb'))

In [93]:
filename = 'testImages/chicken3.jpg'

img_array = imread (filename)
img_resized = resize (img_array, (150, 150, 3))
test_data = np.array (img_resized.flatten())
test_data = test_data.reshape(1, -1)
print (test_data.shape)

(1, 67500)


In [94]:
p = model.predict (test_data)
if p==0: print('chickenHead')
elif p==1: print('elephantHead')

chickenHead


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


## Neural Network

In [23]:
model = MLPClassifier (hidden_layer_sizes = (10,15), activation='relu', max_iter= 250)
model.fit(xtrain, ytrain)

ypred_train = model.predict (xtrain)
ypred_test = model.predict (xtest)

print ('Train Accuracy: ', accuracy_score (ypred_train, ytrain))
print ('Test Accuracy: ', accuracy_score (ypred_test, ytest))

Train Accuracy:  1.0
Test Accuracy:  0.975


In [24]:
## sometimes it returns:
## Train Accuracy: 1.0
## Test Accuracy: 0.975

##and other times
## Train Accuracy: 0.6 
## Test Accuracy: 0.46
## or some other low number

In [25]:
pickle.dump (model, open('NeuralNetworkModel.pkl', 'wb'))

In [26]:
model = pickle.load (open('KNN_Model.pkl', 'rb'))

In [27]:
filename = 'testImages/elephent1.jpg'

img_array = imread (filename)
img_resized = resize (img_array, (150, 150, 3))
test_data = np.array (img_resized.flatten())
test_data = test_data.reshape(1, -1)
print (test_data.shape)

(1, 67500)


In [28]:
p = model.predict (test_data)
if p==0: print('chickenHead')
elif p==1: print('elephantHead')

elephantHead


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
